In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import *   
import load_data
import nan_imputation
import helpers
from helpers import find_repo_root

# Step 0 : load the data from Optogenetics

In [24]:
import importlib
importlib.reload(load_data)
importlib.reload(helpers)

<module 'helpers' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/helpers.py'>

In [22]:
repo_root = find_repo_root()
repo_root

data_path = os.path.join(repo_root, 'Data/Optogenetics')

In [25]:
worms = load_data.load_lifespan(data_path)
#worms.pop("worm_1_companyDrug", None)  # The second argument avoids KeyError if the key doesn't exist

FileNotFoundError: [Errno 2] No such file or directory: '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Optogenetics/companyDrug'

For optogenetics dataset:
Atr- should not react to light
Atr+ should react to light 

Can we predict the response to the light but without the light